In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Data counts (image counts)

In [ ]:
import pandas as pd
import numpy as np
import os
import argparse
import glob

def data_statistics(data_directory):
    """main function for updating the data statistics file in the root moths folder"""
    
    df = pd.DataFrame([], columns=["Family","Genus","Species","N"])
    family_dirs = next(os.walk(data_directory))[1]
    
    for f in family_dirs:
        genus_dirs = next(os.walk(data_directory + f))[1]
        for g in genus_dirs:
            species_dirs = next(os.walk(data_directory + f + '/' + g))[1]
            for s in species_dirs:
                images = os.listdir(data_directory + f + '/' + g + '/' + s)
                images = [k for k in images if 'jpg' in k]
                n = len(images)
                data = [f, g, s, n]
                data = pd.DataFrame(data).transpose()
                data.columns = ["Family","Genus","Species","N"]
                df = pd.concat([df, data], ignore_index=True)
    return df
        
    # print(family_dirs)

df1 = data_statistics('/bask/homes/f/fspo1218/amber/data/gbif_macro_data/gbif_macro/')
df2 = data_statistics('/bask/homes/f/fspo1218/amber/data/binary_moth_training/gbif_crops/')


In [ ]:
df2

In [ ]:
# read in the csv
import pandas as pd
import numpy as np

df = pd.read_csv('/bask/homes/f/fspo1218/amber/data/gbif_macro_data/gbif_macro/data_statistics.csv')
df.head()

In [ ]:
# create a histogram of the counts
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 3))

ax[0].hist(df1['N'], bins=20)
ax[0].set_title('Counts of species images downloaded (All GBIF, 990 species total)')
ax[0].set_xlabel('Number of images collected')
ax[0].set_ylabel('Counts')


ax[1].hist(df2['N'], bins=20)
ax[1].set_title('Counts of species images downloaded (Megadetector cropped, 990 species total)')
ax[1].set_xlabel('Number of images collected')
ax[1].set_ylabel('Counts')

plt.show()


df = pd.read_csv('/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/gbif_images/try_wrapper')

In [ ]:
df = pd.read_csv('/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/gbif_images/try_wrapper/data_statistics.csv')
df.shape

In [ ]:
# create a histogram of the counts
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 3))

ax[0].hist(df['image_count'], bins=20)
ax[0].set_title('Counts of species images downloaded (' + str(df.shape[0]) + ' species total)')
ax[0].set_xlabel('Number of images collected')
ax[0].set_ylabel('Counts')


ax[1].hist(df.loc[df['image_count'] < 100, 'image_count'], bins=20)
ax[1].set_title('Counts of species images downloaded (where number < 100)')
ax[1].set_xlabel('Number of images collected')
ax[1].set_ylabel('Counts')

plt.show()

# Look at predictions

In [ ]:
import pandas as pd
df = pd.read_csv('/bask/projects/v/vjgo8416-amber/projects/on_device_classifier/model_training/updated_myfile.csv', on_bad_lines='skip')

df.head()

In [ ]:
# temp = df['True_label'] == df['MILA_prediction']
# image_species = list(set(df['True_label']))
# mila_species = list(set(df['MILA_prediction']))

import json

f = open("/bask/homes/f/fspo1218/amber/data/mila_models/01-moths-ukdenmark_v2_category_map_species_names(1).json")
label_info = json.load(f)
species_list_mila = list(label_info)
print(len(species_list_mila), " species in total")

f = open("/bask/homes/f/fspo1218/amber/data/binary_moth_training/gbif_crop_ready/01_uk_macro_data_numeric_labels.json")
label_info = json.load(f)
species_list = label_info["species_list"]
print(len(species_list), " species in total")


common = [value for value in species_list_mila if value in species_list]
print(len(common), "species common to both")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


acc_type = ["Top 10", "Top 3", "Top 1"]
accuracies = {
    'TF Lite': (sum(df['TFLite_top10'])/df.shape[0] * 100, sum(df['TFLite_top3'])/df.shape[0] * 100, sum(df['TFLite_top1'])/df.shape[0] * 100),
    'Pytorch': (sum(df['Pytorch_top10'])/df.shape[0] * 100, sum(df['Pytorch_top3'])/df.shape[0] * 100, sum(df['Pytorch_top1'])/df.shape[0] * 100),
    'MILA': (sum(df['MILA_top10'])/df.shape[0] * 100, sum(df['MILA_top3'])/df.shape[0] * 100, sum(df['MILA_top1'])/df.shape[0] * 100),
}

x = np.arange(len(acc_type))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots()#layout='constrained')

for attribute, measurement in accuracies.items():
    print_meas = [round(x, 2) for x in measurement]
    offset = width * multiplier
    rects = ax.bar(x + offset, print_meas, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Percentage of Cases')
ax.set_xlabel('Type')
ax.set_title('Number of cases where the top X predicted species contains the Truth')
ax.set_xticks([x + 0.125 for x in list(range(0, len(acc_type)))])
ax.set_xticklabels(acc_type)
ax.legend()#loc='upper left', ncols=3)
ax.set_ylim(0, 100)

plt.show()

In [ ]:
def calc_acc(true_col, pred_col):
    temp = [1 if df[true_col][i] == df[pred_col][i] else 0 for i in range(df.shape[0])] 
    temp = sum(temp)/df.shape[0] * 100
    return(temp)

In [ ]:
acc_type = ("Species", "Genus", "Family")

accuracies = {
    'TF Lite': (calc_acc('True_label', 'TFLite_prediction'), calc_acc('True_genus', 'TFLite_genus'), calc_acc('True_family', 'TFLite_family')),
    'Pytorch': (calc_acc('True_label', 'Pytorch_prediction'), calc_acc('True_genus', 'Pytorch_genus'), calc_acc('True_family', 'Pytorch_family')),
}

x = np.arange(len(acc_type))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots()#layout='constrained')

for attribute, measurement in accuracies.items():
    print_meas = [round(x, 2) for x in measurement]
    offset = width * multiplier
    rects = ax.bar(x + offset, print_meas, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Percentage of Cases')
ax.set_xlabel('Taxonomic Level')
ax.set_title('Truth = Top 1 at Taxonomic Levels')
ax.set_xticks([x + 0.125 for x in list(range(0, len(acc_type)))])
ax.set_xticklabels(acc_type)
ax.legend()#loc='upper left', ncols=3)
ax.set_ylim(0, 100)

plt.show()

In [ ]:
df.loc[df['True_label_index'] == 0, ]

# Species Heatmap

In [ ]:
df3 = pd.crosstab(df["TFLite_prediction"], df["True_label"], dropna=True, normalize="index")
df3 = df3.fillna(0)


# Force the df to be square
i = list(set(list(df["TFLite_prediction"]) + list(df["True_label"])))
df3 = df3.reindex(index=i, columns=i, fill_value=0)

df3.head()

In [ ]:
import operator

fam_dict = pd.Series(list(df.True_family) + list(df.True_family), index=list(df.True_label)+list(df.TFLite_prediction)).to_dict()
gen_dict = pd.Series(list(df.True_genus) + list(df.True_genus),index=list(df.True_label)+list(df.TFLite_prediction)).to_dict()

In [ ]:
import seaborn as sns

In [ ]:
fam_col = sns.color_palette("husl", len(set(fam_dict.values())))
gen_col = sns.color_palette("husl", len(set(gen_dict.values())))

fam_col = dict(zip(list(set(fam_dict.values())), [tuple(int(c*255) for c in cs) for cs in fam_col]))
gen_col = dict(zip(list(set(gen_dict.values())), [tuple(int(c*255) for c in cs) for cs in gen_col]))

# #colors = [tuple(t / 255 for t in x) for x in rgb]

# fam_col = [fam_col[fam_dict[x]] for x in list(df3.columns)]
# gen_col = [gen_col[gen_dict[x]] for x in list(df3.columns)]

# fam_col = [tuple(round(t / 255, 2) for t in x) for x in fam_col]
# gen_col = [tuple(round(t / 255, 2) for t in x) for x in gen_col]

# len(fam_col)
fam_col

In [ ]:
#lut = {'group_1': 'red', 'group_2': 'blue', 'group_3': 'green'}
row_colors = pd.DataFrame([fam_dict[x] for x in list(df3.columns)]).map(fam_col)
row_colors

In [ ]:
for i, color in enumerate(row_colors):
    ax.add_patch(plt.Rectangle(xy=(-0.05, i), width=0.05, height=1, color=color, lw=0,
                               transform=ax.get_yaxis_transform(), clip_on=False))

In [ ]:
from scipy.cluster.hierarchy import linkage
import matplotlib

link = linkage(df3.transpose()) 

genus_colors = pd.DataFrame(data={'genus': gen_col }) 
family_colors = pd.DataFrame(data={'family': fam_col }) 

sns.set(font_scale=1)

sns.clustermap(df3, row_linkage=link, col_linkage=link, cmap=sns.cm.rocket_r,
               row_colors=genus_colors, col_colors=family_colors, 
               norm=matplotlib.colors.LogNorm(), figsize=(20,15))


plt.show()

# Genus heatmap

In [ ]:
df4 = pd.crosstab(df["TFLite_genus"], df["True_genus"], dropna=True, normalize="index")
df4 = df4.fillna(0)


# Force the df to be square
i = list(set(list(df["TFLite_genus"]) + list(df["True_genus"])))
df4 = df4.reindex(index=i, columns=i, fill_value=0)

df4.head()

In [ ]:
from scipy.cluster.hierarchy import linkage
import matplotlib

link = linkage(df4.transpose()) 

sns.set(font_scale=1)

sns.clustermap(df4, row_linkage=link, col_linkage=link, cmap=sns.cm.rocket_r,
               norm=matplotlib.colors.LogNorm(), figsize=(20,15))


plt.show()

# Family Heatmap

In [ ]:
df5.head()

In [ ]:
df5 = pd.crosstab(df["TFLite_family"], df["True_family"], dropna=True, normalize="index")
df5 = df5.fillna(0)
df5 = df5.div(df5.sum(axis=0), axis=1) # normalise to 1

# Force the df to be square
i = list(set(list(df["TFLite_family"]) + list(df["True_family"])))
df5 = df5.reindex(index=i, columns=i, fill_value=0)

link = linkage(df5.transpose()) 

sns.set(font_scale=1)

sns.clustermap(df5, row_linkage=link, col_linkage=link, cmap=sns.cm.rocket_r,
               norm=matplotlib.colors.LogNorm(), figsize=(7,7))


plt.show()

In [ ]:
df5 = pd.crosstab(df["Pytorch_family"], df["True_family"], dropna=True, normalize="index")
df5 = df5.fillna(0)
df5 = df5.div(df5.sum(axis=0), axis=1) # normalise to 1

# Force the df to be square
i = list(set(list(df["Pytorch_family"]) + list(df["True_family"])))
df5 = df5.reindex(index=i, columns=i, fill_value=0)

link = linkage(df5.transpose()) 

sns.set(font_scale=1)

sns.clustermap(df5, row_linkage=link, col_linkage=link, cmap=sns.cm.rocket_r,
               norm=matplotlib.colors.LogNorm(), figsize=(7,7))


plt.show()